In [1]:
import json
import urllib2
import urllib
import xmltodict
import os
import string
from ebooklib import epub 
from  comFunc import *
from multiprocessing import Pool
import time
import re
import functools

In [2]:
key=''.join(open("key").read().split('\n'))

In [3]:
d="/media/some/sophie-hhd/transmitt/fic/#Calibre Fiction/"
dd="/media/some/sophie-hhd/transmitt/fic/Fictional Literature/"
files=! find "$d" "$dd" -type f -iname "*.epub"
len(files)

8837

In [4]:
#query contains book name and author
# key is the goodread API key
#path is location where the cover image shall be stored
def getMeta(query,key,path):
    urlstring="https://www.goodreads.com/search/index.xml?key="+key+"&q="+query.replace(" ","+")
    #print urlstring
    try:
        url=urllib2.urlopen(urlstring)
        data=xmltodict.parse(url.read())
        if len(data)>0:
            meta=data[u'GoodreadsResponse'][u'search'][u'results'][u'work'][0][u'best_book']
            dictt={}
            
            dictt['title']=str(meta[u'title'])
            dictt['author']=str(meta[u'author'][u'name'])
            fileName=dictt['author']+" - "+dictt['title']
            dictt['imageL']=path+dictt['author']+" - "+dictt['title']+".jpg"
            img=str(meta[u'image_url'])
            dictt['imageR']=img
            #urllib.urlretrieve(img,dictt['imageR'])
            return fileName,dictt
        else:
            return 
    except:
        return 
def getFileMeta(f,key,path):

    fp,fn=os.path.split(f)
    #print fn, fp
    #print(f,fp[fp.rindex('/')+1:])
    tmp=fp.replace("/media/some/sophie-hhd/transmitt/fic/#Calibre Fiction/","")
    author=tmp[:tmp.index("/")]
    ttmp=tmp.replace(author+"/","")
    meta=None
    if ttmp.count('(')>0:
        title=ttmp[:ttmp.rindex("(")]
        #r=''.join(m for m in fp[fp.rindex('/')+1:] if m in string.ascii_letters+" ")
        #r=' '.join(q for q in qr.split() if len(q)>3)
        #m=qr.split()
        qr=title +" "+author
        #print "title: ",title,"author: ",author
        meta=getMeta(qr,key,path)

    if meta!=None:
        m=[fn[:fn.rindex('.')],fp]
        s=[mm for mm in meta]
        return m+s
    else:

        qrr=''.join(m for m in fn if m in string.ascii_letters+"."+" ")
        #print "qrr >>>>>>>",qrr
        qr=qrr[:qrr.rindex('.')]
        qr=' '.join(q.strip() for q in qr.split() if q.count('.')==0 and len(q)>1)
        #print "qr >>>",qr
            #print "title: ",title,"author: ",author
        meta=getMeta(qr,key,path)
        if meta!=None:
            m=[fn[:fn.rindex('.')],fp]
            s=[mm for mm in meta]
            return m+s
        else:
            return

In [5]:
f=files[-2000]
print f
m=getFileMeta(f,key,"/home/some/")
print m


/media/some/sophie-hhd/transmitt/fic/Fictional Literature/EPub Collection/Alistair MacLean/Athabasca - Alistair MacLean.epub
['Athabasca - Alistair MacLean', '/media/some/sophie-hhd/transmitt/fic/Fictional Literature/EPub Collection/Alistair MacLean', 'Alistair MacLean - Athabasca', {'imageR': 'https://d.gr-assets.com/books/1318441615m/1039347.jpg', 'author': 'Alistair MacLean', 'imageL': '/home/some/Alistair MacLean - Athabasca.jpg', 'title': 'Athabasca'}]


In [16]:
titles={}#load_obj('data/titlesImNN')#{}
keys=[]#load_obj('data/titleskeysImNN')#[]

In [17]:
flist=list(filter(lambda x: x[x.rindex('/')+1:x.rindex('.')] not in keys, files))
len(flist),len(keys),len(files)

(8837, 0, 8837)

In [19]:
#titles={}
t0 = time.time()
p = Pool(4)
books=p.map(
    functools.partial(getFileMeta,key=key,path="/home/some/covers/"), flist[:1000])
#p.join()
p.terminate()
p.close()


for book in books:
    cnt=0
    try:
        if book[0] not in keys and len(book)>1:
            titles[book[0]]=book[1:]
    except:
        cnt+=1
print "exception count ",cnt        
print len(list(titles.keys()))
save_obj(titles, 'data/titlesImNNN')
keys=list(titles.keys())
save_obj(keys,'data/titleskeysImNNN')
t1=time.time()
total = t1-t0
print('time: ',total)

exception count  1
4484
('time: ', 279.8155508041382)
